#  Tensorflowとは ①基本的な仕組み

### ■簡単な計算
簡単な計算をさせてみます。TensorFlowの基本的な流れです。

In [37]:
import tensorflow as tf

a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)

sess = tf.Session() 
output = sess.run(add) #何を実行するか入力する
print(output) # 12
sess.close()

12


In [38]:
with tf.Session() as sess:
    output = sess.run(add)
    print(output) # 12

12


比較のためにnumpyでも実施

In [39]:
import numpy as np
a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) # 12

12


グラフはノード（丸い部分）とエッジ（線の部分）で表されます。ノードはオペレーション（操作）を表し、エッジはTensor（多次元配列）を表します。

今回の例では左側の灰色のノードでは定数を作るオペレーション（操作）、右の水色のノードでは足し算のオペレーション（操作）が行われています。エッジは整数値です。

<img src = "./images/b10659507cc836291a26834ace057152.png">

### placeholder

placeholderはデータフローグラフの構築時には値が決まっていないものに使います。  
最初は配列の形だけ定義しておいて後から値を入れて使う空箱のような存在です。学習ごとに違う値が入る入力データや正解データなどに用いられます。  
tf.placeholder | TensorFlow  
サンプルコードをplaceholderを使用したものに書き換えてみます。

In [40]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)

sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7}) #cとdに値を与える
print(output) # 12

12


### valiable

Valiableはplaceholderとは違い、データフローグラフの構築時にも値を持ち、更新を行うものに対して使います。  
学習するパラメータ（重み、バイアス）に用いられます。  
tf.Variable | TensorFlow  
これについては次のテキストで詳しく見ていきます。

### constant
確認になりますが、placeholderでもValiableでもないただの値は定数constantとして扱います。

### セッションは最後に終了する

sess.close()

with tf.Session() as sess:
    sess.run() # ここに計算の実行コードを入れていく

### ロジスティック回帰の実装

### 論理回路
簡単な題材として、ロジスティック回帰による論理回路の再現を行います。  
論理回路は2つの値を入力し、1つの値を出力する関数のようなものです。  
入力も出力も0か1のみで表され、入力される組み合わせによって出力する値が変わります。  
ANDゲートは入力された2つの値が両方とも1だった場合、出力が1となり、それ以外の組み合わせでは0を出力します。
<img src = "./images/24b9632f52dcdd8c226a1be3eccc2361.png" width="400" height="300">



In [41]:
import numpy as np
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [42]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

### フォワード関数とクロスエントロピー誤差の算出式を実装

In [43]:
y = tf.sigmoid(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1-t) * tf.log(1-y))

###### 学習を行うために、勾配降下法を用いてパラメータを最適化するためのコードを加える。
目的関数をGradientDescentOptimizerに渡す
GradientDescentOptimizer()の引数は学習率である。

In [44]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

学習後の結果の正解が正しいかどうかの判定と正解率の計算もデータフローグラフとして定義できます。

In [45]:
correct_prediction = tf.equal(tf.sign(y-0.5), tf.sign(t-0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

1行目で結果が正解かどうか判定しています。一つ一つ見ていきましょう。  
まずtf.equal()は引数に指定された2つの値が等しいかどうかを判定してくれます。  
返り値はBool値です。tf.sign()は引数の値が正なら1、0なら0、負なら-1を返します。  
yが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。  

2行目は正解率を計算するためのコードです。  
tf.reduce_mean()は多次元配列の各成分の平均を計算する関数です。  
tf.cast()でBool値を0,1に変換しています。つまりここでは正解で1、不正解で0と判定された配列の平均値をとっているので正解率を表していることになります。

### データを入力して計算
セッションを準備してパラメータを最適化する計算を行います。

In [46]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [47]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
# 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


最後に各サンプルの計算結果を確認してロジスティック回帰の実装を終わります。  
先ほどのaccuracyと同様、実行することで計算結果が返ってくるためprintします。

In [48]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})

print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]

print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]


[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9651403e-04]
 [4.9049813e-02]
 [4.9049813e-02]
 [9.3120378e-01]]


### 重みとバイアスの最終データを確認

In [49]:
print('W:', sess.run(W))
print('b:', sess.run(b))

W: [[5.569955]
 [5.569955]]
b: [-8.53458]


### 途中の値が見たい場合
デバッグのために途中の値が見たい場合もあります。  
  
例えば、y = tf.sigmoid(tf.matmul(x, W) + b)のtf.matmul(x, W)の部分が見たいといったことを考えます。

出力yがsess.run(y, feed_dict={x:x_train, t:y_train})で見れたことと同様の行えば良いため、次のようなコードになります。

In [50]:
mat = tf.matmul(x, W)
y = tf.sigmoid(mat + b)
print(sess.run(mat, feed_dict={
    x:x_train,
    t:y_train
}))

[[ 0.      ]
 [ 5.569955]
 [ 5.569955]
 [11.13991 ]]


### 最後には必ずセッションの終了を忘れない！！

In [34]:
sess.close()
#もしくは
#with tf.Session() as sess:
#   sess.run() # ここに計算の実行コードを入れていく

# 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）  
* 重みを初期化する必要があった  
* エポックのループが必要だった  
*　隠れ層が複数のモデルを構築するためにクラスで層の情報をまとめる必要があった。

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。  

# 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」が  
TensorFlowではどう実装されているかを確認してください。

それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [51]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 81.6887, val_loss : 35.4289, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 27.5467, val_loss : 60.1035, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 26.7046, val_loss : 15.0704, acc : 0.250, val_acc : 0.625
Epoch 3, loss : 6.8037, val_loss : 13.6479, acc : 0.750, val_acc : 0.375
Epoch 4, loss : 0.0009, val_loss : 0.6120, acc : 1.000, val_acc : 0.938
Epoch 5, loss : 0.0002, val_loss : 0.6884, acc : 1.000, val_acc : 0.938
Epoch 6, loss : 3.3383, val_loss : 5.7413, acc : 0.750, val_acc : 0.500
Epoch 7, loss : 1.3267, val_loss : 1.7666, acc : 0.750, val_acc : 0.875
Epoch 8, loss : 2.8308, val_loss : 5.3688, acc : 0.750, val_acc : 0.625
Epoch 9, loss : 0.5651, val_loss : 1.7316, acc : 0.750, val_acc : 0.875
test_acc : 0.900


# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Iris Species  
サンプルコードと同じくこの中のtrain.csvを使用してください。目的変数はSpeciesに含まれる3種類全てを使います。  
2クラスの分類と3クラス以上の分類の違いを考慮してください。  
それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。  

In [57]:
y_train

array([[1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1]])